# CMIP6 Historical model runs
This script opens up all the CMIP6 historical model run files with data on precipitation pr and surface temperature tas and loops over the files.


In [1]:
import xarray as xr, matplotlib.pyplot as plt
#import sys 
#sys.path.append('') # use this if the function file is in a different directory to the notebook
from importlib import reload # need to use this if I edit a function file
import anomaly_function as func # imports the function
import os
import numpy as np
import pandas as pd
import cartopy.crs as ccrs # to add in continents and change map projections 
from matplotlib.colors import LinearSegmentedColormap # to change colour bar????

In [2]:
# use this to reload the function file if I make any edits to it
func = reload(func) 

In [3]:
!ls /g/data/lp01/CMIP6/CMIP/CAS/CAS-ESM2-0/historical/r1i1p1f1/Amon/pr/gr1.5

v20200302


In [4]:
# store each section of the directory as a string
institution_dir = '/g/data/lp01/CMIP6/CMIP/'
tas_dir = '/historical/r1i1p1f1/Amon/tas/gr1.5/'
pr_dir = '/historical/r1i1p1f1/Amon/pr/gr1.5/'
print(institution_dir, tas_dir, pr_dir)

/g/data/lp01/CMIP6/CMIP/ /historical/r1i1p1f1/Amon/tas/gr1.5/ /historical/r1i1p1f1/Amon/pr/gr1.5/


In [5]:
models_tas = func.read_models(institution_dir, tas_dir, '1850-01','2015-01')

45 model paths found and loaded into the dictionary "models"
MPI-ESM-1-2-HAM /g/data/lp01/CMIP6/CMIP/HAMMOZ-Consortium/MPI-ESM-1-2-HAM/historical/r1i1p1f1/Amon/tas/gr1.5/v20190627/*.nc
NESM3 /g/data/lp01/CMIP6/CMIP/NUIST/NESM3/historical/r1i1p1f1/Amon/tas/gr1.5/v20190630/*.nc
Path for MCM-UA-1-0 does not exist
ACCESS-CM2 /g/data/lp01/CMIP6/CMIP/CSIRO-ARCCSS/ACCESS-CM2/historical/r1i1p1f1/Amon/tas/gr1.5/v20190919/*.nc
Model MPI-ESM1-2-HR has different time so is now removed
MPI-ESM1-2-LR /g/data/lp01/CMIP6/CMIP/MPI-M/MPI-ESM1-2-LR/historical/r1i1p1f1/Amon/tas/gr1.5/v20190710/*.nc
IPSL-CM6A-LR /g/data/lp01/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/historical/r1i1p1f1/Amon/tas/gr1.5/v20180803/*.nc
CAS-ESM2-0 /g/data/lp01/CMIP6/CMIP/CAS/CAS-ESM2-0/historical/r1i1p1f1/Amon/tas/gr1.5/v20200302/*.nc
FGOALS-f3-L /g/data/lp01/CMIP6/CMIP/CAS/FGOALS-f3-L/historical/r1i1p1f1/Amon/tas/gr1.5/v20190927/*.nc
FGOALS-g3 /g/data/lp01/CMIP6/CMIP/CAS/FGOALS-g3/historical/r1i1p1f1/Amon/tas/gr1.5/v20190818/*.nc
BCC-ESM1 

In [6]:
models_pr = func.read_models(institution_dir, pr_dir, '1850-01','2015-01')

45 model paths found and loaded into the dictionary "models"
MPI-ESM-1-2-HAM /g/data/lp01/CMIP6/CMIP/HAMMOZ-Consortium/MPI-ESM-1-2-HAM/historical/r1i1p1f1/Amon/pr/gr1.5/v20190627/*.nc
NESM3 /g/data/lp01/CMIP6/CMIP/NUIST/NESM3/historical/r1i1p1f1/Amon/pr/gr1.5/v20190630/*.nc
Path for MCM-UA-1-0 does not exist
ACCESS-CM2 /g/data/lp01/CMIP6/CMIP/CSIRO-ARCCSS/ACCESS-CM2/historical/r1i1p1f1/Amon/pr/gr1.5/v20190919/*.nc
MPI-ESM1-2-HR /g/data/lp01/CMIP6/CMIP/MPI-M/MPI-ESM1-2-HR/historical/r1i1p1f1/Amon/pr/gr1.5/v20190710/*.nc
MPI-ESM1-2-LR /g/data/lp01/CMIP6/CMIP/MPI-M/MPI-ESM1-2-LR/historical/r1i1p1f1/Amon/pr/gr1.5/v20190710/*.nc
IPSL-CM6A-LR /g/data/lp01/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/historical/r1i1p1f1/Amon/pr/gr1.5/v20180803/*.nc
CAS-ESM2-0 /g/data/lp01/CMIP6/CMIP/CAS/CAS-ESM2-0/historical/r1i1p1f1/Amon/pr/gr1.5/v20200302/*.nc
FGOALS-f3-L /g/data/lp01/CMIP6/CMIP/CAS/FGOALS-f3-L/historical/r1i1p1f1/Amon/pr/gr1.5/v20190927/*.nc
FGOALS-g3 /g/data/lp01/CMIP6/CMIP/CAS/FGOALS-g3/historical/r1i1p

In [7]:
# make sure the models are the same for both vairables
# delete the height from the models_tas array (if it hasn't been deleted already)
if 'height' in models_tas:
    del models_tas['height']
# delete models that aren't common between variables
models_tas = models_tas.where(models_tas.model.isin(models_pr.model), drop = True)
models_pr = models_pr.where(models_pr.model.isin(models_tas.model), drop = True)
#chekc that the models are the same for both variables and if so print True
models_tas.model.equals(models_pr.model)

True

In [8]:
# combine precipitation and temperature into one dataset
combined = xr.Dataset({'tas': models_tas.tas, 'pr': models_pr.pr})
combined

,Array,Chunk
Bytes,9.12 GB,228.10 MB
Shape,"(40, 1980, 120, 240)","(1, 1980, 120, 240)"
Count,325 Tasks,40 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,9.12 GB,228.10 MB
Shape,"(40, 1980, 120, 240)","(1, 1980, 120, 240)"
Count,325 Tasks,40 Chunks
Type,float32,numpy.ndarray


Right now I have all the models stored in one array so from hereafter I can calculate anomalies etc.  

In [215]:
# area weighting 
models_tasw = models_tas*np.cos(models_tas.lat*(np.pi/180))
models_prw = models_pr*np.cos(models_pr.lat*(np.pi/180))

In [216]:
# select out hte 10 year time frame around each eruption (5yrs before and after)
# tas
Krakatau_tas = models_tasw.sel(time = slice('1878-08','1888-08'))
SantaMaria_tas = models_tasw.sel(time = slice('1897-10','1907-10'))
Agung_tas = models_tasw.sel(time = slice('1958-03','1968-03'))
ElChicon_tas = models_tasw.sel(time = slice('1977-04','1987-04'))
Pinatubo_tas = models_tasw.sel(time = slice('1986-06','1996-06'))

# pr
Krakatau_pr = models_tasw.sel(time = slice('1878-08','1888-08'))
SantaMaria_pr = models_tasw.sel(time = slice('1897-10','1907-10'))
Agung_tas = models_tasw.sel(time = slice('1958-03','1968-03'))
ElChicon_pr = models_tasw.sel(time = slice('1977-04','1987-04'))
Pinatubo_pr = models_tasw.sel(time = slice('1986-06','1996-06'))

In [219]:
# select out the northern and sourhern hemisphere regions
Krakatau_tas_SH = Krakatau_tas.sel(lat=slice(-90,0))
Krakatau_pr_SH = Krakatau_pr.sel(lat=slice(-90,0))
Krakatau_tas_NH = Krakatau_tas.sel(lat=slice(0,90))
Krakatau_pr_NH = Krakatau_pr.sel(lat=slice(0,90))

SantaMaria_tas_SH = SantaMaria_tas.sel(lat=slice(-90,0))
SantaMaria_pr_SH = SantaMaria_pr.sel(lat=slice(-90,0))
SantaMaria_tas_NH = SantaMaria_tas.sel(lat=slice(0,90))
SantaMaria_pr_NH = SantaMaria_pr.sel(lat=slice(0,90))

Agung_tas_SH = Agung_tas.sel(lat=slice(-90,0))
Agung_pr_SH = Agung_pr.sel(lat=slice(-90,0))
Agung_tas_NH = Agung_tas.sel(lat=slice(0,90))
Agung_pr_NH = Agung_pr.sel(lat=slice(0,90))

ElChicon_tas_SH = ElChicon_tas.sel(lat=slice(-90,0))
ElChicon_pr_SH = ElChicon_pr.sel(lat=slice(-90,0))
ElChicon_tas_NH = ElChicon_tas.sel(lat=slice(0,90))
ElChicon_pr_NH = ElChicon_pr.sel(lat=slice(0,90))

Pinatubo_tas_SH = Pinatubo_tas.sel(lat=slice(-90,0))
Pinatubo_pr_SH = Pinatubo_pr.sel(lat=slice(-90,0))
Pinatubo_tas_NH = Pinatubo_tas.sel(lat=slice(0,90))
Pinatubo_pr_NH = Pinatubo_pr.sel(lat=slice(0,90))

In [218]:
# import land fraction data
landfrac_ds = xr.open_dataset('/home/563/kb6999/Models/landfraction_file_grid1.5.nc')
landmask = landfrac_ds.mean(dim='time')

In [220]:
# this is where I use the land mask and select out the continents
# and I select the Australia region with a lon,lat box and the Eastern Australia region
Krakatau_tas_aus = Krakatau_tas.where(landmask.data==1, drop=True).sel(lat=slice(-50,-10), lon=slice(110,160)) 
Krakatau_pr_aus = Krakatau_pr.where(landmask.data==1, drop=True).sel(lat=slice(-50,-10), lon=slice(110,160))
Krakatau_tas_EA = Krakatau_tas.where(landmask.data==1, drop=True).sel(lat=slice(-50,-10), lon=slice(140,155))
Krakatau_pr_EA = Krakatau_pr.where(landmask.data==1, drop=True).sel(lat=slice(-50,-10), lon=slice(140,155)) 

SantaMaria_tas_aus = SantaMaria_tas.where(landmask.data==1, drop=True).sel(lat=slice(-50,-10), lon=slice(110,160)) 
SantaMaria_pr_aus = SantaMaria_pr.where(landmask.data==1, drop=True).sel(lat=slice(-50,-10), lon=slice(110,160))
SantaMaria_tas_EA = SantaMaria_tas.where(landmask.data==1, drop=True).sel(lat=slice(-50,-10), lon=slice(140,155))
SantaMaria_pr_EA = SantaMaria_pr.where(landmask.data==1, drop=True).sel(lat=slice(-50,-10), lon=slice(140,155)) 

Agung_tas_aus = Agung_tas.where(landmask.data==1, drop=True).sel(lat=slice(-50,-10), lon=slice(110,160)) 
Agung_pr_aus = Agung_pr.where(landmask.data==1, drop=True).sel(lat=slice(-50,-10), lon=slice(110,160))
Agung_tas_EA = Agung_tas.where(landmask.data==1, drop=True).sel(lat=slice(-50,-10), lon=slice(140,155))
Agung_pr_EA = Agung_pr.where(landmask.data==1, drop=True).sel(lat=slice(-50,-10), lon=slice(140,155)) 

ElChicon_tas_aus = ElChicon_tas.where(landmask.data==1, drop=True).sel(lat=slice(-50,-10), lon=slice(110,160)) 
ElChicon_pr_aus = ElChicon_pr.where(landmask.data==1, drop=True).sel(lat=slice(-50,-10), lon=slice(110,160))
ElChicon_tas_EA = ElChicon_tas.where(landmask.data==1, drop=True).sel(lat=slice(-50,-10), lon=slice(140,155))
ElChicon_pr_EA = ElChicon_pr.where(landmask.data==1, drop=True).sel(lat=slice(-50,-10), lon=slice(140,155)) 

Pinatubo_tas_aus = Pinatubo_tas.where(landmask.data==1, drop=True).sel(lat=slice(-50,-10), lon=slice(110,160)) 
Pinatubo_pr_aus = Pinatubo_pr.where(landmask.data==1, drop=True).sel(lat=slice(-50,-10), lon=slice(110,160))
Pinatubo_tas_EA = Pinatubo_tas.where(landmask.data==1, drop=True).sel(lat=slice(-50,-10), lon=slice(140,155))
Pinatubo_pr_EA = Pinatubo_pr.where(landmask.data==1, drop=True).sel(lat=slice(-50,-10), lon=slice(140,155)) 



In [ ]:

# define an array to store the anomalies for each model
monthly_anom=[]

# loop over each model 
for m in Krakatau_tas.model:
    # group the data into months
    variable_monthly = Krakatau_tas.sel(model=m).groupby('time.month')

    # calculate the mean climatology along each month for the time period 1850-1900 
    clim_monthly = Krakatau_tas.sel(model=m).sel(time[0:57]).groupby('time.month').mean(dim = 'time')

    # caclulate the anomalies for each month and return it as an array
    anom_monthly = (variable_monthly - clim_monthly)

    # append each model into the array
    monthly_anom.append(anom_monthly)

# Combine the individual data sets into a single xarray dataset.
multi_monthly_anom = xr.concat(monthly_anom, dim='model')


In [239]:
# use functions to calculate the monthly anomalies for the globe
K_mon_anom_tas_Glob = func.monthly_anomaly(Krakatau_tas, '1878-08', '1883-05')
K_mon_anom_pr_Glob = func.monthly_anomaly(Krakatau_pr, '1878-08', '1883-05')

S_mon_anom_tas_Glob = func.monthly_anomaly(SantaMaria_tas, '1897-10', '1902-07')
S_mon_anom_pt_Glob = func.monthly_anomaly(SantaMaria_pr, '1897-10', '1902-07')

A_mon_anom_tas_Glob = func.monthly_anomaly(Agung_tas, '1958-03', '1962-12')
A_mon_anom_pr_Glob = func.monthly_anomaly(Agung_pr, '1958-03', '1962-12')

E_mon_anom_tas_Glob = func.monthly_anomaly(ElChicon_tas, '1977-04', '1982-02')
E_mon_anom_pr_Glob = func.monthly_anomaly(ElChicon_pr, '1977-04', '1982-02')

P_mon_anom_tas_Glob = func.monthly_anomaly(Pinatubo_tas, '1986-06', '1991-03')
P_mon_anom_pr_Glob = func.monthly_anomaly(Pinatubo_pr, '1986-06', '1991-03')

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-20.07/lib/python3.7/site-packages/xarray/core/indexing.py:1343: PerformanceWarning: Slicing with an out-of-order index is generating 11 times more chunks
  return self.array[key]
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-20.07/lib/python3.7/site-packages/xarray/core/indexing.py:1343: PerformanceWarning: Slicing with an out-of-order index is generating 11 times more chunks
  return self.array[key]
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-20.07/lib/python3.7/site-packages/xarray/core/indexing.py:1343: PerformanceWarning: Slicing with an out-of-order index is generating 11 times more chunks
  return self.array[key]
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-20.07/lib/python3.7/site-packages/xarray/core/indexing.py:1343: PerformanceWarning: Slicing with an out-of-order index is generating 11 times more chunks
  return self.array[key]
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-20.07/lib/python3.7/site-pack